## Reconhecimento de digito manuscrito

### Preparação dos dados, treino e validação

In [3]:
import numpy as np
import cv2

# Carregar e processar a imagem dos dígitos
image = cv2.imread('digits.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
small = cv2.pyrDown(image)

# Exibir a imagem redimensionada
cv2.imshow('Digits Image', small)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Dividir a imagem em 5000 células, cada uma de 20x20 pixels
# Isso resulta em um array 4D de tamanho 50 x 100 x 20 x 20
cells = [np.hsplit(row, 100) for row in np.vsplit(gray, 50)]

cv2.imshow("digit", cells[0][0])
cv2.waitKey(0)
cv2.destroyAllWindows()

# Converter a lista para um array numpy com forma (50,100,20,20)
x = np.array(cells)
print("The shape of our cells array: " + str(x.shape))

# Dividir o conjunto de dados em dois segmentos
# Um para treinamento e outro para teste
train = x[:, :70].reshape(-1, 400).astype(np.float32)  # Tamanho = (3500, 400)
test = x[:, 70:100].reshape(-1, 400).astype(np.float32)  # Tamanho = (1500, 400)

# Criar labels para os dados de treino e teste
k = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
train_labels = np.repeat(k, 350)[:, np.newaxis]
test_labels = np.repeat(k, 150)[:, np.newaxis]

# Iniciar kNN, treinar os dados e testar com o conjunto de teste para k=3
knn = cv2.ml.KNearest_create() 
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
ret, result, neighbors, distance = knn.findNearest(test, k=3)

# Verificar a acurácia da classificação
matches = result == test_labels
correct = np.count_nonzero(matches)
accuracy = correct * (100.0 / result.size)
print("Accuracy is = %.2f" % accuracy + "%")

The shape of our cells array: (50, 100, 20, 20)
Accuracy is = 93.47%


### Definindo funções úteis

In [4]:
# Função para obter a coordenada x do centróide de um contorno
def x_cord_contour(contour):
    if cv2.contourArea(contour) > 10:
        M = cv2.moments(contour)
        if M['m00'] != 0:
            return int(M['m10'] / M['m00'])
    return 0

# Função para transformar a imagem em um quadrado, adicionando bordas pretas
def makeSquare(not_square):
    BLACK = [0, 0, 0]
    height, width = not_square.shape[:2]
    
    if height == width:
        return not_square
    
    max_dim = max(height, width)
    padding_vert = (max_dim - height) // 2
    padding_horz = (max_dim - width) // 2
    
    square_img = cv2.copyMakeBorder(
        not_square, 
        padding_vert, padding_vert, padding_horz, padding_horz, 
        cv2.BORDER_CONSTANT, value=BLACK
    )
    
    return square_img

# Função para redimensionar a imagem para um tamanho específico
def resize_to_pixel(dimensions, image):
    buffer_pix = 4
    dimensions -= buffer_pix
    
    r = float(dimensions) / image.shape[1]
    new_dim = (dimensions, int(image.shape[0] * r))
    
    resized = cv2.resize(image, new_dim, interpolation=cv2.INTER_AREA)
    
    height_r, width_r = resized.shape[:2]
    BLACK = [0, 0, 0]

    # Padroniza o tamanho final da imagem
    if height_r != width_r:
        diff = abs(height_r - width_r)
        if height_r > width_r:
            resized = cv2.copyMakeBorder(resized, 0, 0, diff // 2, diff // 2, cv2.BORDER_CONSTANT, value=BLACK)
        else:
            resized = cv2.copyMakeBorder(resized, diff // 2, diff // 2, 0, 0, cv2.BORDER_CONSTANT, value=BLACK)

    # Adiciona bordas extras para garantir o tamanho adequado
    ReSizedImg = cv2.copyMakeBorder(resized, 2, 2, 2, 2, cv2.BORDER_CONSTANT, value=BLACK)

    return ReSizedImg


### Carregando uma nova imagem, preprocessando ela e reconhecendo os dígitos

In [ ]:
# Carregar a imagem
image = cv2.imread('numbers.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Mostrar as imagens original e em escala de cinza
cv2.imshow("image", image)
cv2.imshow("gray", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Desfocar a imagem e encontrar as bordas usando Canny
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imshow("blurred", blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

edged = cv2.Canny(blurred, 30, 150)
cv2.imshow("edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Encontrar contornos
contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Ordenar contornos
contours = sorted(contours, key=x_cord_contour, reverse=False)

# Criar um array vazio para armazenar o número completo
full_number = []

# Loop sobre os contornos
for c in contours:
    (x, y, w, h) = cv2.boundingRect(c)

    if w >= 5 and h >= 25:
        roi = blurred[y:y + h, x:x + w]
        ret, roi = cv2.threshold(roi, 127, 255, cv2.THRESH_BINARY_INV)
        cv2.imshow("threshold", roi)
        squared = makeSquare(roi)
        final = resize_to_pixel(20, squared)
        cv2.imshow("final", final)

        if final.shape[0] == 20 and final.shape[1] == 20:
            final_array = final.reshape((1, 400)).astype(np.float32)
            # Usar findNearest
            ret, result, neighbors, dist = knn.findNearest(final_array, k=1)

            # Extraindo o valor do resultado corretamente
            number = str(int(result[0].item()))
            full_number.append(number)

            # Desenhar um retângulo ao redor do dígito
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image, number, (x, y + 155), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)
            cv2.imshow("image", image)
            cv2.waitKey(0)
        else:
            print(f"Dimensões inesperadas para a ROI: {final.shape}")

cv2.destroyAllWindows()
print("The number is: " + ''.join(full_number))
